In [1]:
# I'm adding a version of this code that was used to draft a version of the manuscript. It includes some 
# un-used analyses (like a pretty effect size matrix / heatmap) and probably a lot of attempts to do things.
# If it hasn't been cleaned up before publication, it should be .... 
# Anything good in here was probably written by chatGPT, and anything bad was probably written by me
# npetersen@ucla.edu 08/06/2024


import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager
from matplotlib.patches import Patch
import matplotlib.colors as mcolors
import colorsys
import IPython.display as display

# load in csv version of datasheet and give the full path so you can run this from anywhere
file_path = '/u/home/n/npeterse/TMS_data_copy.csv'
data = pd.read_csv(file_path)

# and stop cutting off data!
pd.set_option('display.max_rows', None)  


# lets make a little dictionary of important variables
# sometimes you want the main dvs, sometimes you want all dvs. maybe that should go inside the relevant cells
# even if redundant

dvs = ['SJW_SUM_C', 'SJW_SUM_PSY', 'SJW_SUM_PHY', 'SJW_SUM_SS', 'SJW_SUM_OVERALL', 'PANAS_POS_TOT', 'PANAS_NEG_TOT', 'UTS_TOT']
experimental_targets = ['PPC', 'SFG', 'DLPFC']
control_target = 'v5'
targets = ['PPC', 'SFG', 'v5', 'DLPFC']
ordered_targets = ['DLPFC', 'SFG', 'PPC', 'v5']
subset_data = data.dropna(subset=dvs + ['PrePost', 'PTID'])
required_columns = ['PANAS_POS_TOT', 'SJW_SUM_OVERALL', 'STITARGET', 'PrePost', 'PTID', 'SJW_SUM_C', 'SJW_SUM_PSY', 'SJW_SUM_PHY', 'SJW_SUM_SS', 'SJW_SUM_A', 'SJW_SUM_OVERALL']
data_clean = data.dropna(subset=required_columns)

target_colors = {
    'SFG': '#1ABC9C', 
    'DLPFC': '#1A8BBC',
    'PPC': '#9563d0', 
    'v5': '#7f7f7f'  
}

# labels for figures:
label_translation = {
    'SJW_SUM_C_PPC': 'Craving (SJ)',
    'SJW_SUM_C_SFG': 'Craving (SJ)',
    'SJW_SUM_C_v5': 'Craving (SJ)',
    'SJW_SUM_C_DLPFC': 'Craving (SJ)',
    'SJW_SUM_PSY_PPC': 'Psych Withdrawal',
    'SJW_SUM_PSY_SFG': 'Psych Withdrawal',
    'SJW_SUM_PSY_v5': 'Psych Withdrawal',
    'SJW_SUM_PSY_DLPFC': 'Psych Withdrawal',
    'SJW_SUM_PHY_PPC': 'Physio Withdrawal',
    'SJW_SUM_PHY_SFG': 'Physio Withdrawal',
    'SJW_SUM_PHY_v5': 'Physio Withdrawal',
    'SJW_SUM_PHY_DLPFC': 'Physio Withdrawal',
    'SJW_SUM_SS_PPC': 'Stim/Sedate',
    'SJW_SUM_SS_SFG': 'Stim/Sedate',
    'SJW_SUM_SS_v5': 'Stim/Sedate',
    'SJW_SUM_SS_DLPFC': 'Stim/Sedate',
    'SJW_SUM_A_PPC': 'Appetite',
    'SJW_SUM_A_SFG': 'Appetite',
    'SJW_SUM_A_v5': 'Appetite',
    'SJW_SUM_A_DLPFC': 'Appetite',
    'SJW_SUM_OVERALL_PPC': 'Withdrawal sum',
    'SJW_SUM_OVERALL_SFG': 'Withdrawal sum',
    'SJW_SUM_OVERALL_v5': 'Withdrawal sum',
    'SJW_SUM_OVERALL_DLPFC': 'Withdrawal sum',
    'PANAS_POS_TOT_PPC': '+Affect',
    'PANAS_POS_TOT_SFG': '+Affect',
    'PANAS_POS_TOT_v5': '+Affect',
    'PANAS_POS_TOT_DLPFC': '+Affect',
    'PANAS_NEG_TOT_PPC': '-Affect',
    'PANAS_NEG_TOT_SFG': '-Affect',
    'PANAS_NEG_TOT_v5': '-Affect',
    'PANAS_NEG_TOT_DLPFC': '-Affect',
    'UTS_TOT_PPC': 'Craving (UTS)',
    'UTS_TOT_SFG': 'Craving (UTS)',
    'UTS_TOT_v5': 'Craving (UTS)',
    'UTS_TOT_DLPFC': 'Craving (UTS)',
}
# uncomment the line below to display the first few rows of the dataframe
#data.head()

In [3]:
# main analysis (time-by-site interaction) goes here

# load in csv version of datasheet
file_path = '/u/home/n/npeterse/TMS_data_copy.csv' 
data = pd.read_csv(file_path)

# list of main dependent variables
dvs = ['SJW_SUM_OVERALL', 'PANAS_NEG_TOT', 'UTS_TOT']

# Fit linear mixed models for interaction between time (Pre/Post) and target for each dependent variable
experimental_targets = ['PPC', 'SFG', 'DLPFC']
control_target = 'v5'
results = []

# Function to compute Cohen's d
def cohen_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / dof)
    return (np.mean(x) - np.mean(y)) / pooled_std

# Models for each dependent variable and each experimental target
for dependent_variable in dvs:
    for target in experimental_targets:
        subset_data = data[data['STITARGET'].isin([target, control_target])]
        # Drop missing values for the relevant dependent variable
        subset_data = subset_data.dropna(subset=[dependent_variable, 'PrePost', 'PTID'])
        if not subset_data.empty:
            # Add a new column to indicate whether the target is experimental or control
            subset_data['TargetGroup'] = subset_data['STITARGET'].apply(lambda x: 'Experimental' if x == target else 'Control')
            model = smf.mixedlm(f"{dependent_variable} ~ PrePost * TargetGroup", subset_data, groups=subset_data["PTID"]).fit()
            interaction_term = 'PrePost[T.Pre]:TargetGroup[T.Experimental]'
            if interaction_term in model.pvalues:
                pre_data = subset_data[(subset_data['PrePost'] == 'Pre') & (subset_data['TargetGroup'] == 'Experimental')][dependent_variable]
                post_data = subset_data[(subset_data['PrePost'] == 'Post') & (subset_data['TargetGroup'] == 'Experimental')][dependent_variable]
                d = cohen_d(pre_data, post_data)
                result = {
                    'Dependent Variable': dependent_variable,
                    'Target': target,
                    'Estimate': model.params[interaction_term],
                    'Standard Error': model.bse[interaction_term],
                    't-value': model.tvalues[interaction_term],
                    'p-value': model.pvalues[interaction_term],
                    'Effect Size (Cohen\'s d)': d,
                    'Random Effects': model.random_effects
                }
                results.append(result)
            else:
                result = {
                    'Dependent Variable': dependent_variable,
                    'Target': target,
                    'Estimate': None,
                    'Standard Error': None,
                    't-value': None,
                    'p-value': None,
                    'Effect Size (Cohen\'s d)': None,
                    'Random Effects': model.random_effects
                }
                results.append(result)

results_df = pd.DataFrame(results)
display.display(results_df)


,Dependent Variable,Target,Estimate,Standard Error,t-value,p-value,Effect Size (Cohen's d),Random Effects
0,SJW_SUM_OVERALL,PPC,1.472956,1.919740,0.767269,0.442922,0.092623,"{37001: [-11.807964519684637], 37003: [-9.7578..."
1,SJW_SUM_OVERALL,SFG,4.514369,1.832638,2.463318,0.013766,0.328632,"{37001: [-12.937425612270097], 37003: [-8.6134..."
2,SJW_SUM_OVERALL,DLPFC,3.416146,1.863153,1.833530,0.066724,0.278033,"{37001: [-11.99604363351965], 37003: [-8.08018..."
3,PANAS_NEG_TOT,PPC,-0.252370,0.948483,-0.266078,0.790179,0.147143,"{37001: [-3.2503963280487196], 37003: [-3.1458..."
4,PANAS_NEG_TOT,SFG,0.651148,1.172228,0.555479,0.578567,0.237328,"{37001: [-3.16629392005981], 37003: [-3.166293..."
5,PANAS_NEG_TOT,DLPFC,0.564292,0.995028,0.567112,0.570638,0.274829,"{37001: [-3.7294681739841464], 37003: [-2.3965..."
6,UTS_TOT,PPC,0.976900,2.237793,0.436546,0.662440,0.097637,"{37001: [-12.461312687473105], 37003: [-2.6520..."
7,UTS_TOT,SFG,4.001760,2.017479,1.983545,0.047307,0.264944,"{37001: [-12.36701490035432], 37003: [-0.80073..."
8,UTS_TOT,DLPFC,3.557875,2.241916,1.586980,0.112517,0.280314,"{37001: [-15.116268150836712], 37003: [0.05712..."


In [6]:
# attempting to clean up the for loops from above without breaking anything... looks good, this will be main loop 
# for post hoc t-tests

# added this to try to get more precise p-values but it didn't work (or did it?)
pd.set_option('display.float_format', lambda x: f'{x:.4g}' if abs(x) < 1e-4 else f'{x:.4f}')

# load in csv version
file_path = '/u/home/n/npeterse/TMS_data_copy.csv' 
data = pd.read_csv(file_path)

# commenting this out to attempt to move dropna inside the main loop
#required_columns = ['PANAS_POS_TOT', 'SJW_SUM_OVERALL', 'STITARGET', 'PrePost', 'PTID', 'SJW_SUM_C', 'SJW_SUM_PSY', 'SJW_SUM_PHY', 'SJW_SUM_SS', 'SJW_SUM_A', 'SJW_SUM_OVERALL']
#data_clean = data.dropna(subset=required_columns)

# list of all dependent variables
dvs = ['SJW_SUM_C', 'SJW_SUM_PSY', 'SJW_SUM_PHY', 'SJW_SUM_SS', 'SJW_SUM_A', 'SJW_SUM_OVERALL', 'PANAS_POS_TOT', 'PANAS_NEG_TOT', 'UTS_TOT']

# Function to compute Cohen's d
def cohen_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    pooled_std = np.sqrt(((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / dof)
    return (np.mean(x) - np.mean(y)) / pooled_std

# Fit linear mixed models for each target separately for PANAS_POS_TOT and SJW subscales
targets = ['PPC', 'SFG', 'v5', 'DLPFC']
results = {}

# Models for each SJW subscale
for dependent_variable in dvs:
    for target in targets:
        subset_data = data_clean[data_clean['STITARGET'] == target]
        # Drop missing values for the relevant dependent variable
        subset_data = subset_data.dropna(subset=[dependent_variable, 'PrePost', 'PTID'])
        if not subset_data.empty:
            model = smf.mixedlm(f"{dependent_variable} ~ PrePost", subset_data, groups=subset_data["PTID"]).fit()
            if 'PrePost[T.Pre]' in model.pvalues:
                pre_data = subset_data[subset_data['PrePost'] == 'Pre'][dependent_variable]
                post_data = subset_data[subset_data['PrePost'] == 'Post'][dependent_variable]
                d = cohen_d(pre_data, post_data)
                results[f'{dependent_variable}_{target}'] = {
                    'p-value': model.pvalues['PrePost[T.Pre]'],
                    'effect_size (Cohen\'s d)': d
                }
            else:
                results[f'{subscale}_{target}'] = {
                    'p-value': None,
                    'effect_size (Cohen\'s d)': None
                }

# Create DataFrame for p-values and effect sizes
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df

,p-value,effect_size (Cohen's d)
SJW_SUM_C_PPC,0.1051,0.1341
SJW_SUM_C_SFG,4.119e-05,0.3600
SJW_SUM_C_v5,0.1051,0.1398
SJW_SUM_C_DLPFC,0.0002,0.3919
SJW_SUM_PSY_PPC,0.0804,0.1990
SJW_SUM_PSY_SFG,0.0246,0.3135
SJW_SUM_PSY_v5,0.9614,-0.0130
SJW_SUM_PSY_DLPFC,0.0479,0.1774
SJW_SUM_PHY_PPC,0.8050,-0.0425
SJW_SUM_PHY_SFG,0.5644,0.0664
